In [1]:
import os
import pandas as pd
from pandas import DataFrame as DF

In [8]:
#build class for read data

class proccess_challenge:
    def __init__(self):
        self.__path = f'{os.getcwd()}/data/'
        self.sc = spark.sparkContext
        self.format = '.parquet'
    
    def read_raw(self, name_file : str):
        #read csv file as rdd
        return self.sc.textFile(f'{self.__path}/raw/{name_file}')
        
    def read_dataset_pd(self, name_file : str) -> None:
            df = pd.read_csv(f'{self.__path}/raw/{name_file}')
            self.save_parquet(name_file, df)
    
    def save_parquet(self, name_file : str, df : DF) -> None:
        df_transform = self.transform_column(df)
        df_transform.to_parquet(f'{self.__path}/results/{name_file}{self.format}')
        
    def transform_column(self, df : DF) -> DF:
        # rename columns the DataFrame
        for file in os.listdir(os.getcwd() + '/data/raw/'):
            df = pd.read_csv(os.getcwd() + '/data/raw/' + file)
            for col in df.columns:
                col_new = col.translate(col.maketrans('/%"_', f'____')).replace(' ', '')
                df.rename(columns={col: col_new.upper()}, inplace=True)
        return df
    
    def main(self):
        for file in os.listdir(f'{self.__path}/raw/'):
            self.read_dataset_pd(file)
            print('file process  {}'.format(file))

In [9]:
obj = proccess_challenge()
obj.main()

file process  worldometer_data.csv
file process  full_grouped.csv
file process  day_wise.csv
file process  covid_19_clean_complete.csv
file process  country_wise_latest.csv
file process  usa_county_wise.csv
